### **Downloding Faker lib and restarting kernal**

In [0]:
#!pip install Faker

In [0]:
#%restart_python

### **Creating fake data for order and order items**

In [0]:
from faker import Faker
import random
import pandas as pd
import time
n = 8
i = 0
while(i < n):
    df_prod = spark.read.format("csv").option("header","True").option("inferSchema","True").load("/Volumes/ecommerce/data/raw_data/olist_products_dataset.csv")
    df_cust = spark.read.format("csv").option("header","True").option("inferSchema","True").load("/Volumes/ecommerce/data/raw_data/olist_customers_dataset.csv")

    df_prod.write.format("delta").mode("overwrite").saveAsTable("dim_products")
    df_cust.write.format("delta").mode("overwrite").saveAsTable("dim_customers")

    faker = Faker()

    products = [row["product_id"] for row in spark.table("dim_products").collect()]
    customer = [row["customer_id"] for row in spark.table("dim_customers").collect()]

    orders = []
    for i in range(50):
        order_id = f"ORD-{i+1}"
        customer_id = random.choice(customer)
        order_date = faker.past_date()
        orders.append((order_id, customer_id, order_date,"delivered"))
        
    orders_df = pd.DataFrame(orders, columns=["order_id", "customer_id", "order_date","status"])
    orders_df.display()
    order_item = []
    for order_id in orders_df["order_id"]:
        for _ in range(random.randint(1,5)):
            product_id = random.choice(products)
            quantity = random.randint(1,5)
            price = round(random.uniform(1,1000),2)
            order_item.append((order_id, product_id, quantity, price))
            
    order_item_df = pd.DataFrame(order_item, columns=["order_id", "product_id", "quantity", "price"])

    fileind = -1
    file_info_list = dbutils.fs.ls("/Volumes/ecommerce/data/streaming_data/Order/")
    file_list = [file_info.name for file_info in file_info_list]
    for filename in file_list:
        if filename.startswith("batch_"):
            currfileind = int(filename.split(".")[0].split("_")[1])
            if currfileind > fileind:
                fileind = currfileind

    fileind += 1
    print(fileind)
    orders_df.to_json(f"/Volumes/ecommerce/data/streaming_data/Order/batch_{fileind}.json", orient="records", lines=True)
    order_item_df.to_json(f"/Volumes/ecommerce/data/streaming_data/Order_items/batch_{fileind}.json", orient="records", lines=True)
    time.sleep(10)
    i+=1




dbutils.fs.rm("/Volumes/ecommerce/data/streaming_data/Order_items/",True)
dbutils.fs.rm("/Volumes/ecommerce/data/streaming_data/order/",True)
dbutils.fs.mkdirs("/Volumes/ecommerce/data/streaming_data/Order_items/")
dbutils.fs.mkdirs("/Volumes/ecommerce/data/streaming_data/Order/")


dbutils.fs.rm("/Volumes/ecommerce/data/streaming_data/Order_items_checkpoints/",True)
dbutils.fs.rm("/Volumes/ecommerce/data/streaming_data/order_checkpoint/",True)
dbutils.fs.mkdirs("/Volumes/ecommerce/data/streaming_data/Order_items_checkpoints/")
dbutils.fs.mkdirs("/Volumes/ecommerce/data/streaming_data/Order_checkpoint/")


In [0]:
from faker import Faker

faker = Faker()

data = faker.past_date()
print(data)